In [117]:
import pandas as pd
import csv
import numpy as np
pd.set_option('display.max_rows', 1000)
from sklearn.metrics import classification_report

gold = pd.read_csv("SM4HHT10\\SM4HHT10\\data\\conll\\conll-spans-validation_ctest.conll",
                   sep="\t", quoting = csv.QUOTE_NONE, encoding="utf-8", dtype='str').fillna('------')
gold2 = gold[gold.Id != "------"].copy()

pred = pd.read_csv("SM4HHT10\\SM4HHT10\\models\\flair\\test.tsv",
                   sep="\t", names=['token', 'gold','pred'], quoting = csv.QUOTE_NONE, encoding="utf-8", dtype='str')


In [122]:
df = pd.concat([gold2, pred], axis=1).dropna()

In [124]:
print(classification_report(df.gold, df.pred, labels=["B-MISC","I-MISC"]))

              precision    recall  f1-score   support

      B-MISC       0.96      0.95      0.95      4267
      I-MISC       0.90      0.79      0.84      1574

   micro avg       0.94      0.90      0.92      5841
   macro avg       0.93      0.87      0.90      5841
weighted avg       0.94      0.90      0.92      5841



In [152]:
#!/usr/bin/env python
# coding: utf-8

import pandas as pd
import csv
import numpy as np
import argparse
from tqdm import tqdm

from sklearn.metrics import classification_report


def peek_stack(stack):
    if stack:
        return stack[-1]    # this will get the last element of stack
    else:
        return None

def merge_spans2(spans=[(113, 125), (126, 137), (185, 194), (196, 219)], words, debug=False):
    
    if debug : print(spans)
    spans = np.asarray(spans)
    stack = []
    merged_words = []
    for s, w in zip(spans.reshape(1,-1)[0], words):
        if debug : print(s)
        if len(stack) == 0:
            stack.append(s)
            merged_words.append(w)
        else:
            if s - peek_stack(stack) == 1:
                stack.pop()
                
            else:
                stack.append(s)
        if debug : print("current stack: ", stack)
    stack = np.asarray(stack)
    return stack.reshape(-1, 2), merged_words
    
#merge_spans2(debug=True)    

# def create_submission_file(args):

path = "SM4HHT10\\SM4HHT10\\data\\socialdisner_v3\\train-valid-txt-files\\validation\\"

validation = pd.read_csv("SM4HHT10\\SM4HHT10\\data\\conll\\conll-spans-validation_ctest.conll",
                         sep='\t',quoting=csv.QUOTE_NONE, encoding='utf-8', dtype="object").dropna()

predictions = pd.read_csv("SM4HHT10\\SM4HHT10\\models\\flair\\test.tsv", sep='\t', names=['word', 'gold', 'pred'],
                         encoding='utf-8').dropna()

validation['pred'] = predictions.pred

print(classification_report(predictions.gold, predictions.pred, labels=['B-MISC', 'I-MISC']))

print('converting to submission format')
with open("SM4HHT10\\SM4HHT10\\submissions\\blah.tsv", "w", encoding="utf-8") as fp:
    print('tweets_id','begin','end','type','extraction',sep='\t', file=fp)
    for i in tqdm(range(1, len(validation.Sent_Id.drop_duplicates()) + 1)):
        info = validation[validation.Sent_Id.astype(int) == i]
#         print(info)
        l = info[info.pred != 'O']
        if len(l) == 0: continue
        docid = l.Doc_Id.values[0]
#         print(docid)
        f = open(f'{path}{docid}.txt', 'r', encoding="utf-8")
        # replace tab with space then strip extra spaces
        text = f.read().replace('\n',' ').strip()
        annoying_chars = ["️","⁩","⁦","2⃣"," ", " ", " ", "⁩", "⁦", "‍", " "]
        for char in annoying_chars:
            text = text.replace(char, '_').strip()
        #print(text)
        #print('*'*20)
        #print(l.Doc_Id.values[0], l.begin.values[0], l.end.values[0], l.pred.values[0])
        spans = [[s,e] for s,e in zip(l.begin.astype(int), l.end.astype(int))]
        words = [w for w in l.Word]
        #print(docid, spans)
        spans = merge_spans2(spans, words)
        #print(spans)
        for span in spans:
            aman = text[span[0]: span[1]]
            for char in annoying_chars:
                aman = aman.replace(char, '_').strip()
            print(docid, span[0], span[1], 'ENFERMEDAD', aman, sep='\t', file=fp)

#         "print("#"*20)
        #break


# create_submission_file(args)

SyntaxError: non-default argument follows default argument (2048959999.py, line 19)

,Id,Doc_Id,Sent_Id,Word,Tag,begin,end,pred
